# The Price is Right

Today we build a more complex solution for estimating prices of goods.

1. Day 2.0 notebook: create a RAG database with our 400,000 training data
2. Day 2.1 notebook: visualize in 2D
3. Day 2.2 notebook: visualize in 3D
4. Day 2.3 notebook: build and test a RAG pipeline with GPT-4o-mini
5. Day 2.4 notebook: (a) bring back our Random Forest pricer (b) Create a Ensemble pricer that allows contributions from all the pricers

Phew! That's a lot to get through in one day!

## PLEASE NOTE:

We already have a very powerful product estimator with our proprietary, fine-tuned LLM. Most people would be very satisfied with that! The main reason we're adding these extra steps is to deepen your expertise with RAG and with Agentic workflows.

## We will go fast today! Hold on to your hat..

In [1]:
# imports

import os
import re
import math
import json
from tqdm import tqdm
import random
from dotenv import load_dotenv
from huggingface_hub import login
import matplotlib.pyplot as plt
import numpy as np
import pickle
from openai import OpenAI
from sentence_transformers import SentenceTransformer
from datasets import load_dataset
import chromadb
from testing import Tester

In [2]:
# environment

load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN', 'your-key-if-not-using-env')

In [3]:
# Log in to HuggingFace

hf_token = os.environ['HF_TOKEN']
login(hf_token, add_to_git_credential=True)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [4]:
# Another import after Logging in to Hugging Face - thank you Trung N.!

from items import Item

In [5]:
openai = OpenAI()

In [6]:
# Load in the test pickle file
# See the section "Back to the PKL files" in the day2.0 notebook
# for instructions on obtaining this test.pkl file

with open('test.pkl', 'rb') as file:
    test = pickle.load(file)

In [7]:
def make_context(similars, prices):
    message = "To provide some context, here are some other items that might be similar to the item you need to estimate.\n\n"
    for similar, price in zip(similars, prices):
        message += f"Potentially related product:\n{similar}\nPrice is ${price:.2f}\n\n"
    return message

In [8]:
def messages_for(item, similars, prices):
    system_message = "You estimate prices of items. Reply only with the price, no explanation"
    user_prompt = make_context(similars, prices)
    user_prompt += "And now the question for you:\n\n"
    user_prompt += item.test_prompt().replace(" to the nearest dollar","").replace("\n\nPrice is $","")
    return [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt},
        {"role": "assistant", "content": "Price is $"}
    ]

In [9]:
DB = "products_vectorstore"

In [10]:
client = chromadb.PersistentClient(path=DB)
collection = client.get_or_create_collection('products')

In [11]:
def description(item):
    text = item.prompt.replace("How much does this cost to the nearest dollar?\n\n", "")
    return text.split("\n\nPrice is $")[0]

In [12]:
description(test[0])

"OEM AC Compressor w/A/C Repair Kit For Ford F150 F-150 V8 & Lincoln Mark LT 2007 2008 - BuyAutoParts NEW\nAs one of the world's largest automotive parts suppliers, our parts are trusted every day by mechanics and vehicle owners worldwide. This A/C Compressor and Components Kit is manufactured and tested to the strictest OE standards for unparalleled performance. Built for trouble-free ownership and 100% visually inspected and quality tested, this A/C Compressor and Components Kit is backed by our 100% satisfaction guarantee. Guaranteed Exact Fit for easy installation 100% BRAND NEW, premium ISO/TS 16949 quality - tested to meet or exceed OEM specifications Engineered for superior durability, backed by industry-leading unlimited-mileage warranty Included in this K"

In [13]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [14]:
def vector(item):
    return model.encode([description(item)])

In [15]:
def find_similars(item):
    results = collection.query(query_embeddings=vector(item).astype(float).tolist(), n_results=5)
    documents = results['documents'][0][:]
    prices = [m['price'] for m in results['metadatas'][0][:]]
    return documents, prices

In [16]:
print(test[1].prompt)

How much does this cost to the nearest dollar?

Motorcraft YB3125 Fan Clutch
Motorcraft YB3125 Fan Clutch Package Dimensions 25.146 cms (L) x 20.066 cms (W) x 15.494 cms (H) Package Quantity 1 Product Type Auto Part Country Of Origin China Manufacturer Motorcraft, Brand Motorcraft, Model Fan Clutch, Weight 5 pounds, Dimensions 10 x 7.63 x 6.25 inches, Country of Origin China, model number Exterior Painted, Manufacturer Part Rank Automotive Automotive Replacement Engine Fan Clutches 583, Domestic Shipping can be shipped within U.S., International Shipping This item can be shipped to select countries outside of the U.S. Learn More, Available October 10, 2007

Price is $225.00


In [17]:
documents, prices = find_similars(test[1])

In [18]:
print(make_context(documents, prices))

To provide some context, here are some other items that might be similar to the item you need to estimate.

Potentially related product:
Four Seasons 78348 New AC Compressor
New Compressor And Clutch Four Seasons 78348 New Nippondenso 10S17F Compressor w/ Clutch Part number 78348 Package Weight 14.55 pounds Package Dimensions 20.574 H x 28.194 L x 18.034 W (centimeters) Manufacturer Four Seasons, Brand Four Seasons, Weight 6 Pounds, Dimensions 11.3 x 7.2 x 7.8 inches, Country of Origin China, model number 78348, Manufacturer Part 78348, Voltage 120 Volts, Rank Automotive Automotive Replacement Air Conditioning Compressors 1383, Domestic Shipping can be shipped within U.S., International Shipping This item can be shipped to select countries outside of
Price is $186.99

Potentially related product:
Motorcraft YH-23 Blower Motor Resistor
Motorcraft YH-23 Blower Motor Resistor Package Dimensions 6.82 L x 2.36 H x 4.63 W (inches) Fit type Vehicle Specific Package Weight 1.83 pounds Country 

In [19]:
print(messages_for(test[1], documents, prices))

[{'role': 'system', 'content': 'You estimate prices of items. Reply only with the price, no explanation'}, {'role': 'user', 'content': 'To provide some context, here are some other items that might be similar to the item you need to estimate.\n\nPotentially related product:\nFour Seasons 78348 New AC Compressor\nNew Compressor And Clutch Four Seasons 78348 New Nippondenso 10S17F Compressor w/ Clutch Part number 78348 Package Weight 14.55 pounds Package Dimensions 20.574 H x 28.194 L x 18.034 W (centimeters) Manufacturer Four Seasons, Brand Four Seasons, Weight 6 Pounds, Dimensions 11.3 x 7.2 x 7.8 inches, Country of Origin China, model number 78348, Manufacturer Part 78348, Voltage 120 Volts, Rank Automotive Automotive Replacement Air Conditioning Compressors 1383, Domestic Shipping can be shipped within U.S., International Shipping This item can be shipped to select countries outside of\nPrice is $186.99\n\nPotentially related product:\nMotorcraft YH-23 Blower Motor Resistor\nMotorcra

In [20]:
def get_price(s):
    s = s.replace('$','').replace(',','')
    match = re.search(r"[-+]?\d*\.\d+|\d+", s)
    return float(match.group()) if match else 0

In [21]:
get_price("The price for this is $99.99")

99.99

In [22]:
from dotenv import load_dotenv
import os
import openai

load_dotenv()  # loads .env file
openai.api_key = os.getenv("OPEN_API_KEY")


In [23]:
# The function for gpt-4o-mini

def gpt_4o_mini_rag(item):
    documents, prices = find_similars(item)
    response = openai.chat.completions.create(
        model="gpt-4o-mini", 
        messages=messages_for(item, documents, prices),
        seed=42,
        max_tokens=5
    )
    reply = response.choices[0].message.content
    return get_price(reply)

In [24]:
gpt_4o_mini_rag(test[1])

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [ ]:
pip install transformers accelerate bitsandbytes torch


In [ ]:
pip install accelerate


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch

model_name = "microsoft/Phi-3-mini-4k-instruct"  # small, fast model — you can swap this

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load model in 4-bit precision
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto",
    load_in_4bit=True
)

# Create text-generation pipeline
hf_model = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer
)

def gpt_4o_mini_rag(item):
    # Step 1: Retrieve similar docs and prices
    documents, prices = find_similars(item)

    # Step 2: Build your prompt
    messages = messages_for(item, documents, prices)
    prompt = "\n".join([msg["content"] for msg in messages])

    # Step 3: Generate model response
    response = hf_model(
        prompt,
        max_new_tokens=30,
        temperature=0.7,
        do_sample=True
    )

    # Step 4: Extract reply and price
    reply = response[0]["generated_text"]
    return get_price(reply)


In [25]:
test[1].price

225.11

In [26]:
Tester.test(gpt_4o_mini_rag, test)

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

## Optional Extra: Trying a DeepSeek API call instead of OpenAI

If you have a DeepSeek API key, we will use it here as an alternative implementation; otherwise skip to the next section..

In [49]:
# Connect to DeepSeek using the OpenAI client python library

deepseek_api_key = os.getenv("DEEPSEEK_API_KEY")
deepseek_via_openai_client = OpenAI(api_key=deepseek_api_key,base_url="https://api.deepseek.com")

In [50]:
# Added some retry logic here because DeepSeek is very oversubscribed and sometimes fails..

def deepseek_api_rag(item):
    documents, prices = find_similars(item)
    retries = 8
    done = False
    while not done and retries > 0:
        try:
            response = deepseek_via_openai_client.chat.completions.create(
                model="deepseek-chat", 
                messages=messages_for(item, documents, prices),
                seed=42,
                max_tokens=8
            )
            reply = response.choices[0].message.content
            done = True
        except Exception as e:
            print(f"Error: {e}")
            retries -= 1
    return get_price(reply)

In [51]:
deepseek_api_rag(test[1])

Error: Error code: 401 - {'error': {'message': 'Authentication Fails, Your api key: ****-env is invalid', 'type': 'authentication_error', 'param': None, 'code': 'invalid_request_error'}}
Error: Error code: 401 - {'error': {'message': 'Authentication Fails, Your api key: ****-env is invalid', 'type': 'authentication_error', 'param': None, 'code': 'invalid_request_error'}}
Error: Error code: 401 - {'error': {'message': 'Authentication Fails, Your api key: ****-env is invalid', 'type': 'authentication_error', 'param': None, 'code': 'invalid_request_error'}}
Error: Error code: 401 - {'error': {'message': 'Authentication Fails, Your api key: ****-env is invalid', 'type': 'authentication_error', 'param': None, 'code': 'invalid_request_error'}}
Error: Error code: 401 - {'error': {'message': 'Authentication Fails, Your api key: ****-env is invalid', 'type': 'authentication_error', 'param': None, 'code': 'invalid_request_error'}}
Error: Error code: 401 - {'error': {'message': 'Authentication Fa

UnboundLocalError: cannot access local variable 'reply' where it is not associated with a value

In [52]:
Tester.test(deepseek_api_rag, test)

Error: Error code: 401 - {'error': {'message': 'Authentication Fails, Your api key: ****-env is invalid', 'type': 'authentication_error', 'param': None, 'code': 'invalid_request_error'}}
Error: Error code: 401 - {'error': {'message': 'Authentication Fails, Your api key: ****-env is invalid', 'type': 'authentication_error', 'param': None, 'code': 'invalid_request_error'}}
Error: Error code: 401 - {'error': {'message': 'Authentication Fails, Your api key: ****-env is invalid', 'type': 'authentication_error', 'param': None, 'code': 'invalid_request_error'}}
Error: Error code: 401 - {'error': {'message': 'Authentication Fails, Your api key: ****-env is invalid', 'type': 'authentication_error', 'param': None, 'code': 'invalid_request_error'}}
Error: Error code: 401 - {'error': {'message': 'Authentication Fails, Your api key: ****-env is invalid', 'type': 'authentication_error', 'param': None, 'code': 'invalid_request_error'}}
Error: Error code: 401 - {'error': {'message': 'Authentication Fa

UnboundLocalError: cannot access local variable 'reply' where it is not associated with a value

## And now to wrap this in an "Agent" class

In [ ]:
from agents.frontier_agent import FrontierAgent

In [ ]:
# Let's print the logs so we can see what's going on

import logging
root = logging.getLogger()
root.setLevel(logging.INFO)

In [ ]:
agent = FrontierAgent(collection)

In [ ]:
agent.price("Quadcast HyperX condenser mic for high quality podcasting")

In [ ]:
from agents.specialist_agent import SpecialistAgent

In [ ]:
agent2 = SpecialistAgent()

In [ ]:
agent2.price("Quadcast HyperX condenser mic for high quality podcasting")